In [1]:
# Imports
import argparse
import os
import sys
import time
import glob
import re

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from _datetime import datetime
import scipy.stats as stats
import matplotlib.animation as animation
from matplotlib import rcParams
from PIL import Image
from sklearn.utils import resample
from scipy.stats import pearsonr
import scipy.stats as stats
import scipy

In [2]:
# Local imports
sys.path.append("/home/users/benhutch/skill-maps")
import dictionaries as dicts

sys.path.append("/home/users/benhutch/skill-maps/python")
import functions as fnc

In [3]:
# Imports of historical functions
#/home/users/benhutch/skill-maps-differences/functions.py
import os

path = "/home/users/benhutch/skill-maps-differences"
# print(os.listdir(path))

# append path and import
sys.path.append(path)
import functions_diff as hist_fnc

# # List the functions in the module
# print(dir(hist_fnc))

['.git', '.gitignore', 'README.md', 'functions_diff.py', 'dictionaries.py', 'plotting.ipynb', '__pycache__', 'plotting_functions.py', '.ipynb_checkpoints', 'skill_maps_functions.py']
['CDOException', 'CDO_PY_VERSION', 'Cdo', 'CdoTempfileStore', 'Image', 'StringIO', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'animation', 'argparse', 'auto_doc', 'calculate_annual_mean_anomalies_obs', 'calculate_anomalies', 'calculate_correlation_coefficient', 'calculate_correlations', 'calculate_correlations_diff', 'calculate_dcpp_data_ws', 'calculate_historical_data_ws', 'calculate_spatial_correlations', 'calculate_spatial_correlations_bootstrap', 'calculate_spatial_correlations_bootstrap_diff', 'calculate_spatial_correlations_diff', 'calculate_wind_speed', 'ccrs', 'constrain_years_init_uninit', 'constrain_years_processed_hist', 'da', 'dask', 'datetime', 'dd', 'dic', 'extract_ensemble_members', 'extract_historical_data', 'getCdoVersion', 'gl

In [4]:
# Set up the parameters
# Test run with psl dcppA-hindcast raw data and historical data
variable = "tas"
obs_var_name = "tas"
region = "global"
region_grid = dicts.gridspec_global
forecast_range = "2-9"
season = "DJFM"
model_season = "DJFM"
start_year = "1960"
end_year = "2022"

# Set up the paths
observations_path = dicts.obs
dcpp_models = dicts.common_models_noCMCC
historical_models = dicts.rsds_models_historical
base_dir = dicts.base_dir
base_dir_historical = dicts.base_dir_historical
output_dir = dicts.plots_dir
save_dir = dicts.save_dir

In [5]:
%tb
# Process the observations
obs = fnc.process_observations(variable, region, region_grid, forecast_range, season,
                                observations_path, obs_var_name, plev=None)

Gridspec file: /home/users/benhutch/gridspec/gridspec-global.txt
Variable is not ua or va, creating new file name
File already exists
Loading ERA5 data


No traceback available to show.


Dataset loaded:  [[[[33304760. 33304760. 33304760. ... 33304760. 33304760. 33304760.]
   [35274832. 35198748. 35127032. ... 35211220. 35253004. 35267348.]
   [34147296. 34167876. 34185336. ... 34681128. 34786524. 34506512.]
   ...
   [       0.        0.        0. ...        0.        0.        0.]
   [       0.        0.        0. ...        0.        0.        0.]
   [       0.        0.        0. ...        0.        0.        0.]]

  [[      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan       nan ...       nan       nan       nan]
   ...
   [      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan       nan ...       nan       nan       nan]]]


 [[[18759028. 18759028. 18759028. ... 18759028. 18759028. 18759028.]
   [19720678. 19716312. 19719430. ... 19523608. 19593456. 19660808

In [6]:
obs

<xarray.DataArray 'ssrd' (time: 65, lat: 72, lon: 144)>
dask.array<truediv, shape=(65, 72, 144), dtype=float64, chunksize=(8, 72, 100), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1959-12-31 1960-12-31 ... 2023-12-31
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 80.0 82.5 85.0 87.5

In [7]:
if variable in ['rsds', 'ssrd']:
    print("converting from J/m2 to W/m2")
    obs = obs / 86400

convert from J/m2 to W/m2


In [8]:
# Load the historical data
hist_data = hist_fnc.load_processed_historical_data(base_dir_historical, historical_models,
                                                    variable, region, forecast_range, season)

# Extract the data for the variable from the historical data
hist_data, _ = hist_fnc.extract_historical_data(hist_data, variable)

processing model:  BCC-CSM2-MR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/rsds/BCC-CSM2-MR/global/years_2-9/JJA/outputs/processed/*.nc
processing model:  MPI-ESM1-2-HR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/rsds/MPI-ESM1-2-HR/global/years_2-9/JJA/outputs/processed/*.nc
processing model:  CanESM5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/rsds/CanESM5/global/years_2-9/JJA/outputs/processed/*.nc
processing model:  CMCC-CM2-SR5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/rsds/CMCC-CM2-SR5/global/years_2-9/JJA/outputs/processed/*.nc
processing model:  HadGEM3-GC31-MM
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/rsds/HadGEM3-GC31-MM/global/years_2-9/JJA/outputs/processed/*.nc
processing model:  EC-Earth3
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/rsds/EC-Earth3/global/years_2-9/JJA/outputs/processed/*.nc
processing model:  M

In [9]:
# Load the DCPP data
dcpp_data = fnc.load_data(base_dir, dcpp_models, variable, region, 
                            forecast_range, model_season, level=None)

# Extract the data for the variable from the DCPP data
dcpp_data, _ = fnc.process_data(dcpp_data, variable)

Skipping file /home/users/benhutch/skill-maps-processed-data/rsds/EC-Earth3/global/years_2-9/ULG/outputs/mergetime/mergetime_EC-Earth3_rsds_global_2-9_ULG-r8i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/rsds/EC-Earth3/global/years_2-9/ULG/outputs/mergetime/mergetime_EC-Earth3_rsds_global_2-9_ULG-r6i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/rsds/EC-Earth3/global/years_2-9/ULG/outputs/mergetime/mergetime_EC-Earth3_rsds_global_2-9_ULG-r9i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/rsds/EC-Earth3/global/years_2-9/ULG/outputs/mergetime/mergetime_EC-Earth3_rsds_global_2-9_ULG-r10i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/rsds/EC-Earth3/global/years_2-9/ULG/outputs/mergetime/mergetime_EC-Earth3_rsds_global_2-9_ULG-r7i2.nc


In [10]:
# Now we want to make sure that the time period 
# Use constrain years to make sure that the model data is the same
constrained_hist_data = fnc.constrain_years(hist_data, historical_models)

# Constrain the years for the DCPP data
constrained_dcpp_data = fnc.constrain_years(dcpp_data, dcpp_models)

# Align the forecasts and observations
fcst1, fcst2, obs, common_years = fnc.align_forecast1_forecast2_obs(constrained_dcpp_data, dcpp_models, 
                                                                    constrained_hist_data, historical_models, 
                                                                    obs)

shape of obs years pre Nan removal (65,)
NaN values in the obs data for this year:  1959
1959 all NaN values for this year in the obs
NaN values in the obs data for this year:  1960
1960 all NaN values for this year in the obs
NaN values in the obs data for this year:  1961
1961 all NaN values for this year in the obs
NaN values in the obs data for this year:  1962
1962 all NaN values for this year in the obs
1963 no NaN values for this year in the obs
1964 no NaN values for this year in the obs
1965 no NaN values for this year in the obs
1966 no NaN values for this year in the obs
1967 no NaN values for this year in the obs
1968 no NaN values for this year in the obs
1969 no NaN values for this year in the obs
1970 no NaN values for this year in the obs
1971 no NaN values for this year in the obs
1972 no NaN values for this year in the obs
1973 no NaN values for this year in the obs
1974 no NaN values for this year in the obs
1975 no NaN values for this year in the obs
1976 no NaN val

In [ ]:
# Now that we have the forecast1 array for the dcpp data
# We should be able to perform the lagging to quadruple the number of ensemble 
# members
# NOTE: Would it be worth lagging the historical members?
# Set up the lag
lag = 4

# extract the no_members
n_members = fcst1.shape[0] ; n_years = fcst1.shape[1]

# Extract the no_lats
n_lats = fcst1.shape[2] ; n_lons = fcst1.shape[3]

# Set up the no_lagged_members
n_lagged_members = n_members * lag

# Set up the lagged ensemble
lagged_ensemble = np.zeros([n_lagged_members, n_years, n_lats, n_lons])

# Loop over the ensemble members
for member in range(n_members):
    # Loop over the years
    for year in range(n_years):
        # If the year is less than the lag
        if year < lag - 1:
            # Set the lagged ensemble member equal to NaN
            lagged_ensemble[member, year, :, :] = np.nan

            # Also set the lagged ensemble member equal to NaN
            for lag_i in range(lag):
                lagged_ensemble[member + (lag_i * n_members), year, :, :] \
                = np.nan
        # Otherwise
        else:
            # Loop over the lag
            for lag_i in range(lag):
                # Set the lagged ensemble member equal to the forecast
                lagged_ensemble[member + (lag_i * n_members), year, :, :] = \
                    fcst1[member, year - lag_i, :, :]
                
# Now we have the lagged ensemble
# The first 3 years of the lagged ensemble should be NaN
# Remove these years
lagged_ensemble = lagged_ensemble[:, lag-1:, :, :]

# Do the same for the observations
lagged_obs = obs[lag-1:, :, :]

# Do the same for the second forecast
lagged_fcst2 = fcst2[:, lag-1:, :, :]

# Check the shape of the lagged ensemble
if lagged_ensemble[0].shape == obs.shape:
    print("lagged ensemble and observations have the same shape")
else:
    raise ValueError("lagged ensemble and observations do not have the same" +
                        "shape")

# Check the shape of the lagged ensemble
if lagged_fcst2[0].shape == obs.shape:
    print("lagged ensemble and observations have the same shape")
else:
    raise ValueError("lagged ensemble and observations do not have the same" +
                        "shape")

In [ ]:
# Load up the data for the NAO matched members

In [12]:
# Now we can test the bootstrapping function
forecast_stats_raw = fnc.forecast_stats(obs, fcst1, fcst2, no_boot=10)

bootstrap index 0
shape of obs_boot (45, 72, 144)
value of obs_boot [[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 5.66360532  6.20386429  6.12671369 ...  6.07562066  5.94156901
    6.00949978]
  [ 4.24652705  3.93631149  3.89064417 ...  4.29845414  4.4922866
    4.49233073]
  [ 2.68602051  2.71219455  2.72397045 ...  2.43226002  2.34866211
    2.51464699]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 3.41969907  4.10251302  4.26025174 ...  3.71201534  3.65585974
    3.74875217]
  [ 2.03901259  1.62503816  1.45576136 ...  2.10778139  2.35274161
    2.34255986]
  [-0.79607883 -0.70073134 -0.63121139 ... -1.08111744

In [13]:
# process the forecast stats for the lagged ensemble
forecast_stats_lagged = fnc.forecast_stats(lagged_obs, lagged_ensemble, 
                                            lagged_fcst2, no_boot=10)

{'corr1': array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [0.51331272, 0.54403432, 0.54812514, ..., 0.58840733, 0.54480468,
        0.52989605],
       [0.30633439, 0.27842244, 0.29847085, ..., 0.34387112, 0.34059382,
        0.33245234],
       [0.22729118, 0.23861865, 0.24867063, ..., 0.19272982, 0.17833131,
        0.20252902]]), 'corr1_min': array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [ 0.18348989,  0.18211674,  0.15375806, ...,  0.25829093,
         0.20143739,  